In [1]:
from autoscope import Autoscope

In [3]:
scope = Autoscope([12, 16, 20, 21], [6, 13, 19, 26], [4, 17, 27, 22])

In [32]:
scope.focus.up(500)

In [25]:
scope.stage.right(2000)

In [33]:
scope.coordinates

(0, 0, 17500)

In [25]:
stage.home()

In [34]:
from picamera import PiCamera
from time import sleep
from io import BytesIO
from PIL import Image
from IPython import display

with PiCamera() as camera:
    camera.resolution = (1280, 720)
    stream = BytesIO()
    for image in camera.capture_continuous(stream, format='jpeg'):
        stream.truncate()
        stream.seek(0)    
        image = Image.open(stream)
        display.clear_output(wait=True)
        display.display(image)

ModuleNotFoundError: No module named 'picamera'

--2018-08-27 21:48:33--  https://gist.githubusercontent.com/python-friends/b51f4deb7d3e35ff516d1682460c3588/raw/GPIO.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.96.133
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.96.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216 [text/plain]
Saving to: ‘RPi/GPIO.py’

RPi/GPIO.py         100%[=====================>]     216  --.-KB/s   in 0s     

Last-modified header missing -- time-stamps turned off.
2018-08-27 21:48:34 (1.21 MB/s) - ‘RPi/GPIO.py’ saved [216/216]



In [17]:
Ggp.cleanup()

clean
